In [101]:
import pandas as pd
import geopandas as gp
import os
import numpy as np

In [24]:
clip_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Clipped"
raw_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw"
project_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected"

In [21]:
# set project CRS
crs = 2926

# TODO: set clipping extent


In [5]:
# clip to boundary
buildings = gp.read_file(os.path.join(raw_path,"/Building_Outlines_2015/Building_Outlines_2015.shp"))

In [26]:
# project to crs

dir_list = os.listdir(clip_path)

for f in dir_list:
    if f[-4:] == '.shp':
        gdf = gp.read_file(os.path.join(clip_path,f))
        print(f)
        gdf_project = gdf.to_crs(crs)
        gdf.to_file(os.path.join(project_path,f'{f[:-4]}_projected.shp'))

contours_5ft_clipped.shp
buildings_2015_clipped.shp
seattle_streets_clipped.shp
SND_streets_clipped.shp


STREET ANALYSIS

In [29]:
# find average street width by functional class
#### TODO- should this be within our zone or across the whole data set?

streets = gp.read_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Seattle_Streets/Seattle_Streets.shp")


In [104]:
streets.SURFACEWID.value_counts()
print('number of roads with no width: ', len(streets[streets.SURFACEWID == 0]))

streets['ARTCLASS'] = streets['ARTCLASS'].map(str)

# backfill data by average width for each functional class
streets_wid = streets[['ARTCLASS','SURFACEWID']]

# remove rows = 0 before finding average
streets_wid = streets_wid[streets_wid.SURFACEWID != 0]
streets_wid.SURFACEWID.value_counts()

# calculate average by arterial class and surface type
streets_wid['ARTCLASS'] = streets_wid['ARTCLASS'].map(str)
streets_wid_avg = streets_wid.groupby(['ARTCLASS']).mean()

## histogram of artclass = 0 surface width
# IQR of not designated roads with a surface width ranges from 22-25. min = 8 and max = 84
# streets[streets['ARTCLASS']=='0.0'][streets['SURFACEWID']!= 0]['SURFACEWID'].describe()

number of roads with no width:  3633


In [107]:
streets_clipped = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Clipped/seattle_streets_clipped.shp"

streets_clipped_gdf = gp.read_file(streets_clipped)
streets_clipped_gdf['ARTCLASS'] = streets_clipped_gdf['ARTCLASS'].map(str)

# backfill data in the CLIPPED AND PROJECTED FILE by average width for each functional class
# NOTE: arterial class == 9 has one feature with surface width == 0. there is no backfilled data for this road
streets_bfill = pd.merge(left = streets_clipped_gdf, right = streets_wid_avg, how = 'left', left_on = 'ARTCLASS', right_on = 'ARTCLASS')
streets_bfill[['OBJECTID','ARTCLASS','UNITDESC','ARTDESCRIP','SURFACEWID_x','SURFACEWID_y','geometry']]
streets_bfill['surface_wid_upd'] = np.where(streets_bfill['SURFACEWID_x'] == 0, streets_bfill['SURFACEWID_y'], streets_bfill['SURFACEWID_x'])

In [108]:
# buffer by average width by arterial class
# NOTE: this could also be offset in rhino? if we just want them to be lines
streets_buffer = streets_bfill.buffer(distance=streets_bfill['surface_wid_upd'])

/tmp/ipykernel_431/3478708111.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  streets_buffer = streets_bfill.buffer(distance=streets_bfill['surface_wid_upd'])


In [109]:
## looking at SND network to compare against seattle streets
SND = gp.read_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Street_Network_Database_(SND)-20230505T014240Z-001/Street_Network_Database_(SND)/Street_Network_Database_(SND).shp")

In [112]:
# TODO: need to look up what the arterial class widths are
SND.columns
SND['ARTERIAL_C'].value_counts()

ARTERIAL_C
0    24818
1     8482
6      382
5      369
4      216
Name: count, dtype: int64

BUILDING ANALYSIS

In [115]:
buildings = gp.read_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/buildings_2015_clipped_projected.shp")

In [123]:
buildings.columns
# 5k buildings do not have building heights
# buildings['BP99_APEX'].value_counts()

# scatter plot of area x building apex for grouping and backfilling
buildings['SHAPE_Area'].describe()

count     34838.000000
mean       1515.829439
std        7203.228198
min           0.092071
25%         153.964832
50%         491.794649
75%        1424.510160
max      319801.892776
Name: SHAPE_Area, dtype: float64